In [1]:
import os
import shutil
import torch
import torch.nn as nn
import numpy as np
import binarybrain
import binarybrain as bb
import torchvision
import torchvision.transforms as transforms
from binarybrain.frame_buffer import FrameBuffer


In [2]:
# configuration
net_name              = 'MnistStochasticLutResnet'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist'
rtl_module_name       = 'MnistLutResnet'
output_velilog_file   = os.path.join(data_path, net_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')
epochs                = 4
mini_batch_size       = 32
bin_mode              = True
frame_modulation_size = 3


# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)


In [3]:
bin_dtype = bb.DType.BIT if bin_mode else bb.DType.FP32


def ResidualXorBlock(lut1, lut2, input_buf):
    b1 = bb.RealToBinary(frame_modulation_size=frame_modulation_size, bin_dtype=bin_dtype)
    b1.send_command("device cuda false")
    conv_block = bb.Sequential([
        bb.Convolution2d(lut1, filter_size=(3, 3), padding='same'),
        bb.Convolution2d(lut2, filter_size=(3, 3), padding='same'),
        b1
    ])

    
   
    conv_block.set_input_shape([1, 28, 28])
    conv_block.send_command("device cuda false")
    
    print(conv_block.get_info())

    # if bin_mode:
    #     conv_block.send_command("binary true")
    # else:
    #     conv_block.send_command("binary false")
    
    #print("conv_block.get_model_list(): ", conv_block.get_model_list())
    #print("get_parameters: ............................  ")
    #a= conv_block.dumps()
    #print(a)
    #print(type(bb.Identity()))
    #print(type(bb.RealToBinary()))
    
    b2= bb.RealToBinary(frame_modulation_size=frame_modulation_size, bin_dtype=bin_dtype)
    b2.send_command("device cuda false")

    identity_block = bb.Sequential([
        bb.Identity(),
        bb.Convolution2d(lut2, filter_size=(3,3), padding='same'),
        b2
    ])

    identity_block.set_input_shape([1, 28, 28])
    identity_block.send_command("device cuda false")
    print(identity_block.get_info())

    # if bin_mode:
    #     identity_block.send_command("binary true")
    # else:
    #     identity_block.send_command("binary false")

    #print("conv_block type: ", type(conv_block))
    #print("identity_block type: ", type(identity_block))

    #a=conv_block.get_core()
    #print("core_mode:", a)
    
    print("type(input_buf)): ", type(input_buf))
    
    conv_input_buffer = conv_block.forward(input_buf)      # FrameBuffer
    print("conv_input_buffer: ", type(conv_input_buffer))

    #conv_output_buffer = bb.FrameBuffer.from_core(conv_input_buffer)
    #print(type(conv_output_buffer))
    identity_input_buffer = identity_block.forward(input_buf)  # FrameBuffer
    print("identity_input_buffer: ", type(identity_input_buffer))
    
    #identity_output_buffer = bb.FrameBuffer.from_core(identity_input_buffer)
    #print(type(identity_output_buffer))
    print(bb.XorPrintBuffer(conv_input_buffer,identity_input_buffer))
    return bb.Xor(conv_input_buffer,identity_input_buffer)
    

In [ ]:
# define residual network(resnet)

lut_layer0_0 = bb.DifferentiableLut([192], batch_norm=False, binarize=False)
print(lut_layer0_0)

lut_layer0_0.send_command("device cuda false")

lut_layer0_1 = bb.DifferentiableLut([32], batch_norm=False, binarize=False)
print(lut_layer0_1)
lut_layer0_1.send_command("device cuda false")


#out1 = lut_layer0_0.forward(in1, train=True)
# img, label= loader_train[0]
# print(img, label)
for images, labels in loader_train:
    # x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    #np_img = np_img.reshape(( 1, 28, 28)) # 1 sample, 1 channel, 28x28
    input_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    print("input_buf node shape: ", input_buf.get_node_shape())


    xor_result = ResidualXorBlock(lut_layer0_0, lut_layer0_1, input_buf)
    print("xor_result: ", xor_result)

    # Check type
    print("XOR buffer dtype (should be BIT):", xor_result.get_type())
    
    # Get numpy array and print first frame's first 10 node values
    xor_numpy = xor_result.numpy()
    print("XOR numpy shape:", xor_numpy.shape)
    print("Sample XOR output values (frame 0):", xor_numpy[0, :10])

 
#net.set_input_shape([1, 28, 28])

# net.send_command("binary false")      
# net.send_command("lut_binarize true")  

#print(net.get_info())


----------------------------------------------------------------------
[DifferentiableLut6] 
 input  shape : {} output shape : {192}
 connection : random
 binary : 0 batch_norm : 0
----------------------------------------------------------------------

----------------------------------------------------------------------
[DifferentiableLut6] 
 input  shape : {} output shape : {32}
 connection : random
 binary : 0 batch_norm : 0
----------------------------------------------------------------------



In [ ]:
image, label= dataset_train[0]

lut_layer= bb.DifferentiableLut([192],batch_norm=False, binarize=False)
lut_layer.set_input_shape([1,28,28])
out = lut_layer.forward(image, train=True)
print(out.get_type())





In [35]:
print(image.shape)


torch.Size([1, 28, 28])


In [10]:
# 534 template<> class DataType<float>
# 535 {
# 536 public:
# 537     typedef float value_type;
# 538     enum {
# 539         type = BB_TYPE_FP32,
# 540         size = 4,
# 541         bit_size = 32,
# 542     };
# 543     static inline std::string Name(void) { return "fp32"; }
# 544 };


SyntaxError: can't use starred expression here (2172759255.py, line 1)

In [8]:
net = bb.Sequential([
    bb.Identity()
])

print(net.get_info())
# net.save('identity_test.bb')

# net2 = bb.Sequential()
# net2.load('identity_test.bb')
# print(net2.get_info())


----------------------------------------------------------------------
[Sequential] 
 input  shape : [] output shape : []
  --------------------------------------------------------------------
  [PassThrough] 
   input  shape : {} output shape : {}
----------------------------------------------------------------------

